In [25]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq

# 1. Yeh line .env file se saari keys load kar degi
load_dotenv() 

# 2. .env se key uthao
groq_key = os.getenv("GROQ_API_KEY")

# 3. Check karo key mil rahi hai ya nahi (sirf testing ke liye)
if not groq_key:
    print("Bhai, .env file se key nahi mili! Check karo file name sahi hai na?")
else:
    # 4. Ab model initialize karein
    llm = ChatGroq(
        groq_api_key=groq_key, 
        model_name="llama3-8b-8192",
        temperature=0
    )
    print("Groq Model Ready!")

Groq Model Ready!


In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
if os.environ.get('GROQ_API_KEY'):
    print("Groq API Key is set.")
else:
    print("Groq API Key NOT found. Check your .env file.")

Groq API Key is set.


In [3]:
from langchain_groq import ChatGroq

# Model initialize karein
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

# Test karne ke liye ek simple message bhejein
messages = [
    ("system", "You are a helpful assistant."),
    ("human", "Hi, kaise ho?"),
]

ai_msg = llm.invoke(messages)
print(ai_msg.content)

c:\Users\drish\OneDrive\Documents\Rag-Implementation\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Main theek hu, dhanyavad. Aap kaise hain? (I'm fine, thank you. How are you?)


## RAG Implementation with My Own Text Data

### Step 1 :Preparing Document for Text

In [11]:
from langchain_core.documents import Document

my_text = """ Cosmetics are substances that are intended for application to the body for cleansing, beautifying, promoting attractiveness, or altering appearance. They are mixtures of chemical compounds derived from either natural sources or created synthetically.[1] Cosmetics have various purposes, including personal and skin care. They can also be used to conceal blemishes and enhance natural features (such as the eyebrows and eyelashes). Makeup can add color to the face, enhance features, or alter appearance to resemble a different person, creature, or object.[2]

People have used cosmetics for thousands of years for skin care and appearance enhancement.[3] Visible cosmetics for both women and men have gone in and out of fashion over the centuries.

Some early cosmetics contained harmful ingredients, such as lead, which caused serious health problems and sometimes resulted in death. Modern commercial cosmetics are generally tested for safety but may contain controversial ingredients, such as per- and polyfluoroalkyl substances (PFAS), formaldehyde releasers, and ingredients that cause allergic reactions.

The European Union and regulatory agencies around the world have stringent regulations for cosmetics. In the United States, cosmetic products and ingredients do not require FDA approval, although marketed products are monitored for safety. Some countries have banned using animal testing for cosmetics."""

doc = Document(page_content=my_text, metadata={"source": "cosmetics.txt"})

doc


Document(metadata={'source': 'cosmetics.txt'}, page_content=' Cosmetics are substances that are intended for application to the body for cleansing, beautifying, promoting attractiveness, or altering appearance. They are mixtures of chemical compounds derived from either natural sources or created synthetically.[1] Cosmetics have various purposes, including personal and skin care. They can also be used to conceal blemishes and enhance natural features (such as the eyebrows and eyelashes). Makeup can add color to the face, enhance features, or alter appearance to resemble a different person, creature, or object.[2]\n\nPeople have used cosmetics for thousands of years for skin care and appearance enhancement.[3] Visible cosmetics for both women and men have gone in and out of fashion over the centuries.\n\nSome early cosmetics contained harmful ingredients, such as lead, which caused serious health problems and sometimes resulted in death. Modern commercial cosmetics are generally tested 

### Step 2 :Spliting the Document into Chunks

In [14]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1. Setup the splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

# 2. THE FIX: Wrap 'doc' in brackets [ ] to make it a list
chunks = text_splitter.split_documents([doc])

# 3. Print results
print(f"Total chunks created: {len(chunks)}")
print("-" * 30)
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1} length: {len(chunk.page_content)}")
    print(f"Content: {chunk.page_content[:100]}...") # Printing first 100 chars
    print("-" * 30)

Total chunks created: 5
------------------------------
Chunk 1 length: 491
Content: Cosmetics are substances that are intended for application to the body for cleansing, beautifying, p...
------------------------------
Chunk 2 length: 116
Content: add color to the face, enhance features, or alter appearance to resemble a different person, creatur...
------------------------------
Chunk 3 length: 187
Content: People have used cosmetics for thousands of years for skin care and appearance enhancement.[3] Visib...
------------------------------
Chunk 4 length: 361
Content: Some early cosmetics contained harmful ingredients, such as lead, which caused serious health proble...
------------------------------
Chunk 5 length: 302
Content: The European Union and regulatory agencies around the world have stringent regulations for cosmetics...
------------------------------


### Step 3 :Creating Embeddings for the Chunks

In [16]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# 1. Embedding Model load karein
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# 2. Chunks ko numbers (Vectors) mein convert karke FAISS mein save karein
# Make sure 'chunks' variable Step 2 se aa raha ho
vector_store = FAISS.from_documents(chunks, embedding_model)

print("Mubarak ho! Vector Database ban gaya hai.")

# 3. Retriever banayein (Search ke liye)
retriever = vector_store.as_retriever(search_kwargs={"k": 2})

Mubarak ho! Vector Database ban gaya hai.


In [18]:
embedding_model.embed_query("Why we use cosmetics?")

[-0.031145170331001282,
 0.06797382235527039,
 0.014410503208637238,
 0.01638200134038925,
 0.09515348821878433,
 0.04116595908999443,
 0.11878076940774918,
 0.03971599414944649,
 0.0015876193065196276,
 -0.011794418096542358,
 0.06788182258605957,
 -0.037095196545124054,
 -0.08797536790370941,
 -0.0247989259660244,
 0.09549937397241592,
 0.015723763033747673,
 0.05142409726977348,
 0.009515638463199139,
 -0.02402820996940136,
 0.013869879767298698,
 -0.002266461495310068,
 -0.03321440517902374,
 0.005694597493857145,
 0.017795471474528313,
 -0.05892380326986313,
 0.013131392188370228,
 -0.057666078209877014,
 -0.033065203577280045,
 0.06224268302321434,
 -0.0434064082801342,
 0.06917580962181091,
 0.10454659909009933,
 0.04649388790130615,
 -0.06337257474660873,
 -0.10711773484945297,
 -0.008945834822952747,
 0.03419419005513191,
 0.013350676745176315,
 -0.018464284017682076,
 0.0005367335397750139,
 -0.08687736839056015,
 -0.1190129965543747,
 -0.10814996808767319,
 0.025483386591076

### Step 4 :Create and Store Embeddings in Vector Store (Storage)

In [19]:
from langchain_community.vectorstores import FAISS

# STEP 4: Create and Store Embeddings in Vector Store
# 'chunks' wo hain jo RecursiveCharacterTextSplitter se banaye the 
# 'embedding_model' wo hai jo load kiya tha 

vectorstore = FAISS.from_documents(
    documents=chunks, 
    embedding=embedding_model
)

print("Vector Store (FAISS) successfully created!")

Vector Store (FAISS) successfully created!


### Step 5: Retrieve  Relevant Information

In [20]:
# User ka sawal
query = "Why do people use cosmetics?"

# Vector store se top 3 relevant chunks nikalein
retrieved_docs = vectorstore.similarity_search(query, k=3)

# Check karne ke liye prints
print(f"Total retrieved chunks: {len(retrieved_docs)}")
for i, doc in enumerate(retrieved_docs):
    print(f"\nChunk {i+1}:")
    print(doc.page_content)

Total retrieved chunks: 3

Chunk 1:
People have used cosmetics for thousands of years for skin care and appearance enhancement.[3] Visible cosmetics for both women and men have gone in and out of fashion over the centuries.

Chunk 2:
Cosmetics are substances that are intended for application to the body for cleansing, beautifying, promoting attractiveness, or altering appearance. They are mixtures of chemical compounds derived from either natural sources or created synthetically.[1] Cosmetics have various purposes, including personal and skin care. They can also be used to conceal blemishes and enhance natural features (such as the eyebrows and eyelashes). Makeup can add color to the face, enhance features, or alter

Chunk 3:
Some early cosmetics contained harmful ingredients, such as lead, which caused serious health problems and sometimes resulted in death. Modern commercial cosmetics are generally tested for safety but may contain controversial ingredients, such as per- and polyfluo

### Step 6: Final Answer Generate Karna (RAG)

In [27]:
# 1. Model ko update karein (llama-3.3-70b-versatile is fast and active)
llm = ChatGroq(
    groq_api_key=os.getenv("GROQ_API_KEY"), 
    model_name="llama-3.3-70b-versatile",
    temperature=0
)

# 2. Context taiyar karein (Step 5 wale chunks)
context_text = "\n\n".join([doc.page_content for doc in retrieved_docs])

# 3. Prompt setup
query = "Why do people use cosmetics?"
prompt = f"""
Answer the question based only on the context below.
Context: {context_text}

Question: {query}
Answer:"""

# 4. Final Result
try:
    response = llm.invoke(prompt)
    print("\n--- Final RAG Answer ---")
    print(response.content)
except Exception as e:
    print(f"Error aaya: {e}")


--- Final RAG Answer ---
People use cosmetics for skin care and appearance enhancement, including cleansing, beautifying, promoting attractiveness, or altering appearance. They can also be used to conceal blemishes and enhance natural features.
